In [193]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [194]:
df = pd.read_csv(r"C:\Users\phunk\Desktop\MyProject\dataanalyst\Cafe Sales\Data\dirty_cafe_sales.csv")

In [195]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [197]:
for col in df.columns:
    print(f"{col} : {df[col].unique()}")

Transaction ID : ['TXN_1961373' 'TXN_4977031' 'TXN_4271903' ... 'TXN_5255387' 'TXN_7695629'
 'TXN_6170729']
Item : ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'UNKNOWN' 'Sandwich' nan
 'ERROR' 'Juice' 'Tea']
Quantity : ['2' '4' '5' '3' '1' 'ERROR' 'UNKNOWN' nan]
Price Per Unit : ['2.0' '3.0' '1.0' '5.0' '4.0' '1.5' nan 'ERROR' 'UNKNOWN']
Total Spent : ['4.0' '12.0' 'ERROR' '10.0' '20.0' '9.0' '16.0' '15.0' '25.0' '8.0' '5.0'
 '3.0' '6.0' nan 'UNKNOWN' '2.0' '1.0' '7.5' '4.5' '1.5']
Payment Method : ['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' 'ERROR' nan]
Location : ['Takeaway' 'In-store' 'UNKNOWN' nan 'ERROR']
Transaction Date : ['2023-09-08' '2023-05-16' '2023-07-19' '2023-04-27' '2023-06-11'
 '2023-03-31' '2023-10-06' '2023-10-28' '2023-07-28' '2023-12-31'
 '2023-11-07' 'ERROR' '2023-05-03' '2023-06-01' '2023-03-21' '2023-11-15'
 '2023-06-10' '2023-02-24' '2023-03-25' '2023-01-15' '2023-04-04'
 '2023-03-30' '2023-12-01' '2023-09-18' '2023-06-03' '2023-12-13'
 '2023-04-20' '2023

In [198]:
df.columns[df.isin(["UNKNOWN", "ERROR"]).any()]

Index(['Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method',
       'Location', 'Transaction Date'],
      dtype='object')

In [199]:
df = df.replace(["UNKNOWN", "ERROR"],np.nan)

In [200]:
df.columns[df.isin(["UNKNOWN", "ERROR"]).any()]

Index([], dtype='object')

In [201]:
df.isnull().sum()

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [202]:
cat_col = ["Item","Payment Method","Location","Transaction Date"]
num_col = ["Quantity","Price Per Unit","Total Spent"]

In [203]:
encode = OrdinalEncoder()
df[cat_col] = encode.fit_transform(df[cat_col])

In [204]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,1.0,2,2.0,4.0,1.0,1.0,250.0
1,TXN_4977031,0.0,4,3.0,12.0,0.0,0.0,135.0
2,TXN_4271903,2.0,4,1.0,NaN,1.0,0.0,199.0
3,TXN_7034554,4.0,2,5.0,10.0,NaN,NaN,116.0
4,TXN_3160411,1.0,2,2.0,4.0,2.0,0.0,161.0


In [205]:
imputer = KNNImputer(n_neighbors=5)
df[cat_col] = imputer.fit_transform(df[cat_col]).round().astype(int)
df[num_col] = imputer.fit_transform(df[num_col])

In [206]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [207]:
for col in cat_col:
    print(f"{col} : {df[col].unique()}")

Item : [1 0 2 4 6 5 3 7]
Payment Method : [1 0 2]
Location : [1 0]
Transaction Date : [250 135 199 116 161  89 278 300 208 364 310 214 122 151  79 318 160  54
  83  14  93  88 334 260 153 346 109  99  69 172 152 309 226 164 281 147
 197 118 158 179 106 355   9 274  53  80 306  60 176 121 247   7  73 328
 338  77 177 108 279 272 146 321 292 275 299  95  30 341 169 347 196  52
 165 342 107 301 119  91 143  70 227 252  65 218 231 104 205 302 348 192
 181  55  92 280 361 241  33 254 123  51  74  36  87 168 265  13 256 258
  97 352 194 345   4  22  50 339 150 222 245 191 156  17  81   3 173 184
 305 211 261  39 246 140 182 190 324 335  71 223  46 100  84 304 202 206
  58  26  18  96  78 360 298  48 134 343 110  34 315 216 129 210 195  10
 273 115 236  59 316 189 132 157 137  16 264 233 207 363 354 270 319 103
   2  11 242 187 285 257 293 244 162 230   5 126 148 141 326 287 317 329
 350 128 294 180 114  49 344 284  27 276 253  56 283  44 117 248 112  21
  68   8 336 217 362  45 144 303  57  

In [208]:
df[cat_col] = encode.inverse_transform(df[cat_col])

In [210]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,5.8,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Credit Card,In-store,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [211]:
df.to_excel("cafe_sales.xlsx",index=False)